In [91]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
# Define the two coplanar debris
a = 3000+6371 << u.km
ecc = 0 << u.one
inc = 0 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 0 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 8000+6371 << u.km
nu = 45 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

In [93]:
print(deb_1.epoch.iso)
print(deb_2.epoch.iso)
print(otv.epoch.iso)


2000-01-01 12:00:00.000
2000-01-01 12:00:00.000
2000-01-01 12:00:00.000


In [98]:
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [113]:
# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas


Target Delta:  32.80801345156272 deg
current_delta :  45.0 deg
t_1:  645.7992464701244 s
orbit_i.nu:  25.751951597504984 deg
new delta:  32.808013451562736 deg
rv:  (array([8440304.52384641, 4071474.00148127,       0.        ]), array([-2833.62024681,  5874.19145482,     0.        ]))
rot_matrix:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
dv_a:  [-284.13119818  589.0136684     0.        ]
dv_b:  [ 255.17061128 -528.9773836     0.        ]


Number of impulses: 2, Total cost: 1.241270 km / s

In [109]:
hoh_phas.impulses

((<Quantity 614.52365025 s>,
  <Quantity [-271.24393117,  595.05829054,    0.        ] m / s>),
 (<Quantity 6435.93934258 s>,
  <Quantity [ 243.59690229, -534.40589669,    0.        ] m / s>))

In [114]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [106]:
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))

OTV ANGLE:  -119.59945479314466 deg
DEB2 ANGLE:  -148.0588510206548 deg


In [78]:
180-1.27



178.73